<a href="https://colab.research.google.com/github/yeesem/Time-Series-Data-Analysis/blob/main/Walk_Forward_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -nc https://lazyprogrammer.me/course_files/airline_passengers.csv

--2023-11-05 01:57:27--  https://lazyprogrammer.me/course_files/airline_passengers.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2036 (2.0K) [text/csv]
Saving to: ‘airline_passengers.csv’

airline_passengers. 100%[===================>]   1.99K  --.-KB/s    in 0s      

2023-11-05 01:57:28 (36.9 MB/s) - ‘airline_passengers.csv’ saved [2036/2036]



In [2]:
!pip install -U statsmodels

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import itertools

In [4]:
 from sklearn.metrics import mean_squared_error
 from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [5]:
df = pd.read_csv('/content/airline_passengers.csv',index_col = 'Month',parse_dates = True)

In [6]:
df.index.freq = 'MS'

In [7]:
df.shape

(144, 1)

In [8]:
#Assume the forecast horizon we care about is 12
#Validate over 10 steps
h = 12
steps = 10
Ntest = len(df) - h - steps + 1

In [9]:
#Configuration hyperparameters to try
trend_type_list = ['add','mul']
seasonal_type_list = ['add','mul']
damped_trend_list = [True,False]
init_method_list = ['estimated','heuristic','legacy-heuristic']
use_boxcox_list = [True,False,0]

In [60]:
def walkforward(
    trend_type,
    seasonal_type,
    damped_trend,
    init_method,
    use_boxcox,
    debug=False):

    #Store errors
    errors = []
    seen_last = False
    steps_completed = 0

    for end_of_train in range(Ntest,len(df)-h+1):
        train = df.iloc[:end_of_train]
        test = df.iloc[end_of_train:end_of_train + h]

        if test.index[-1] == df.index[-1]:
          seen_last = True

        steps_completed += 1

        #Train the model
        hw = ExponentialSmoothing(
          train['Passengers'],
          initialization_method = init_method,
          trend = trend_type,
          damped_trend = damped_trend,
          seasonal = seasonal_type,
          seasonal_periods = 12,
          use_boxcox = use_boxcox
        )
        res_hw = hw.fit()

        #Compute error for the forecast horizon
        fcast = res_hw.forecast(h)
        error = mean_squared_error(test['Passengers'],fcast)
        errors.append(error)

    if debug:
        print('seen_last: ', seen_last)
        print('Steps completed: ',steps_completed)

    return np.mean(errors)

In [61]:
#Test our function
walkforward('add','add',False,'legacy-heuristic',0,debug = True)

seen_last:  True
Steps completed:  10


1448.5344452151644

In [ ]:
#Iterate through all possible options
tuple_of_option_lists = (
    trend_type_list,
    seasonal_type_list,
    damped_trend_list,
    init_method_list,
    use_boxcox_list
)

#Show all the combination
for x in itertools.product(*tuple_of_option_lists):
  print(x)

In [67]:
#Set best score to infinity
best_score = float('inf')
#Set best options to none
best_option = None

for x in itertools.product(*tuple_of_option_lists):
    score = walkforward(*x)

    if score < best_score:
        print("Best score so far : ",score)
        best_score = score
        best_option = x


Best score so far :  412.8172974447663
Best score so far :  397.5872563558714
Best score so far :  368.7875802314191
Best score so far :  320.66417039593273
Best score so far :  308.1359546249236


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:83: RuntimeWarning: overflow 

Best score so far :  305.6593349312611
Best score so far :  299.8202726489487
Best score so far :  261.8795073548492
Best score so far :  249.57507607273482


/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:83: RuntimeWarning: overflow encountered in matmul
  return err.T @ err
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/holtwinters/model.py:83: RuntimeWarning: overflow 

In [68]:
print("Best Score : ",best_score)

trend_type,seasonal_type,damped_trend,init_method,use_boxcox = best_option
print("trend_type    : ", trend_type)
print("seasonal_type : ",seasonal_type)
print("damped_trend  : ",damped_trend)
print("init_method   : ",init_method )
print("use_boxcox    : ",use_boxcox)

Best Score :  249.57507607273482
trend_type    :  mul
seasonal_type :  add
damped_trend  :  False
init_method   :  legacy-heuristic
use_boxcox    :  False
